In [2]:
import pandas as pd
import missingno as msno

from functions import *
from plots import *

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
data_path = r'C:\Users\alejandro.gc\Desktop\Proyectos\Accidentalidad\Accidentes_VALENCIA\data\CLEAN_DATA\clean_data.xlsx'

In [4]:
hojas = ["Accidentes", "Vias", "Tramos", "Personas", "Vehiculos"]
dfs = {}
for hoja in hojas:
    dfs[hoja] = pd.read_excel(data_path, sheet_name=hoja)

df_accidentes = dfs["Accidentes"]
df_vias = dfs["Vias"]
df_tramos = dfs["Tramos"]
df_personas = dfs["Personas"]
df_vehiculos = dfs["Vehiculos"]

In [5]:
df_accidentes['factores concurrentes'] = df_accidentes['factores concurrentes'].str.split('; ')

causas_unicas = set(causa for causas in df_accidentes['factores concurrentes'] for causa in causas)

In [6]:
# Se crean columnas para cada causa única y se asignan valores binarios
for causa in causas_unicas:
    df_accidentes[causa] = df_accidentes['factores concurrentes'].apply(lambda x: 1 if causa in x else 0)

# Eliminar la columna original si ya no es necesaria
df_accidentes = df_accidentes.drop(columns=['factores concurrentes'])

In [7]:
estandarizar_columnas(df_accidentes);

In [8]:
lista_causas = ['cansancio o sueno', 'estado o condicion de la senalizacion',
                'adelantamiento antirreglamentario', 'mal estado del vehiculo',
                'conduccion temeraria', 'meteorologia adversa', 'conduccion distraida',
                'no mantener intervalo de seguridad', 'enfermedad', 'desconocido',
                'drogas', 'tramo en obras', 'estado o condicion de la via',
                'irrumpir animal en calzada', 'alcohol','irrupcion peaton en la calzada', 
                'no respetar la prioridad', 'inexperiencia conductor', 'averia mecanica', 
                'velocidad inadecuada','obstaculo en la calzada', 'conduccion negligente', 
                'otro factor', 'giro incorrecto']

## Datos de personas y vehículos

- El objetivo de este apartado es buscar posibles relaciones/correclaciones que puedan existir entre las personas implicadas en los accidentes y los vehículos que estas conducían o en los que se encontraban como pasajeros.
- Además, se va asignar el tipo de víctima en función de las lesiones sufridas

In [9]:
# Se analizan los tipos de lesiones existentes
df_personas["lesividad"].value_counts()

lesividad
Se desconoce                                                 4030
Sin asistencia sanitaria                                     1302
Ingreso inferior o igual a 24 horas                           804
Atención en urgencias sin posterior ingreso                   627
Asistencia sanitaria inmediata en centro de salud o mutua     252
Ingreso superior a 24 horas                                   219
Asistencia sanitaria solo en el lugar del accidente           198
Asistencia sanitaria ambulatoria con posterioridad            173
Fallecido 24 horas                                             51
Name: count, dtype: int64

In [10]:
# Se definen las agrupaciones de víctimas en función del tipo de lesión
mortales = ['Fallecido 24 horas']

graves = ['Ingreso superior a 24 horas']

leves = ['Ingreso inferior o igual a 24 horas', 'Atención en urgencias sin posterior ingreso', 
         'Asistencia sanitaria inmediata en centro de salud o mutua', 'Asistencia sanitaria solo en el lugar del accidente',
         'Asistencia sanitaria ambulatoria con posterioridad']

ilesos = ["Sin asistencia sanitaria"]

se_desconoce = ["Se desconoce"]

In [11]:
# Se crean nuevas columnas con valores booleanos para definir el tipo de vehículo
tipos_victimas = {
    'mortal': mortales,
    'grave': graves,
    'leve': leves,
    'ileso': ilesos,
    'se desconoce': se_desconoce
}
clasificar_víctimas(df_personas, tipos_victimas, "lesividad");

df_personas['tipo de victima'] = df_personas.apply(asignar_tipo_victima, axis=1)

In [12]:
lista_del_cols = ["lesividad", "mortal", "grave", "leve", "ileso", "se desconoce"]

drop_columns_df(df_personas, lista_del_cols)

## Datos de conductores y tipo de vehículos

In [13]:
# Nos centraremos simplemente en los conductores
mask_conductores = df_personas["posicion"] == "Conductor"

df_conductores = df_personas[mask_conductores]

In [14]:
df_conductores.isna().sum()

codigo del accidente                  0
identificador                         0
presunto responsable                  0
presunto error                        0
vehiculo                              0
posicion                              0
posicion en el vehiculo               0
sexo                                  0
edad                                 51
motivo de desplazamiento              0
desplazamiento previsto               0
tipo de permiso                       0
caracteristica del permiso            0
factor de atencion                    0
signos alcohol                        0
tasa de alcohol 1                     0
signos drogas                         0
casco                                 0
cinturon de seguridad                 0
accion del peaton                     0
presunta infraccion del conductor     0
presunta infraccion de velocidad      0
presunta infraccion del peaton        0
bicicleta                             0
camion                                0


In [15]:
# Se eliminana los nulos que hay en la columna de edad porque no hay un tratamiento obvio
# Antes que imputar la moda o la media, es mejor eliminar dichos valores para no introducir
# variaciones en la distribución y el comportamiento de los conductores.

df_conductores = df_conductores.dropna()

In [16]:
# Se elimanan una serie de columnas, las cuales se van a considerar irrelevantes para 
# este modelo/caso de estudio.

lista_del_cols = ["codigo del accidente", "identificador", "vehiculo", "posicion", "edad", "tasa de alcohol 1"]

drop_columns_df(df_conductores, lista_del_cols)

In [17]:
df_conductores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5611 entries, 0 to 7655
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   presunto responsable               5611 non-null   object
 1   presunto error                     5611 non-null   object
 2   posicion en el vehiculo            5611 non-null   object
 3   sexo                               5611 non-null   object
 4   motivo de desplazamiento           5611 non-null   object
 5   desplazamiento previsto            5611 non-null   object
 6   tipo de permiso                    5611 non-null   object
 7   caracteristica del permiso         5611 non-null   object
 8   factor de atencion                 5611 non-null   object
 9   signos alcohol                     5611 non-null   object
 10  signos drogas                      5611 non-null   object
 11  casco                              5611 non-null   object
 12  cinturon de

In [18]:
df_conductores["rango alcohol"].value_counts()

rango alcohol
No prueba/no positivo    5042
De 0.51 a 1 mg/l          324
De 0.26 a 0.5 mg/l        123
Más de 1 mg/l              57
Inferior a 0.15 mg/l       35
De 0.16 a 0.25 mg/l        30
Name: count, dtype: int64

In [19]:
numeric_cols = df_conductores.select_dtypes(['number']).columns
numeric_cols

Index(['bicicleta', 'camion', 'furgoneta', 'maquinaria obra agricola',
       'motocicleta', 'turismo', 'vmp', 'otro'],
      dtype='object')

In [20]:
vehicle_columns = numeric_cols
df_conductores = agrupar_vehiculo(df_conductores, vehicle_columns)

In [21]:
df_conductores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5611 entries, 0 to 7655
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   presunto responsable               5611 non-null   object
 1   presunto error                     5611 non-null   object
 2   posicion en el vehiculo            5611 non-null   object
 3   sexo                               5611 non-null   object
 4   motivo de desplazamiento           5611 non-null   object
 5   desplazamiento previsto            5611 non-null   object
 6   tipo de permiso                    5611 non-null   object
 7   caracteristica del permiso         5611 non-null   object
 8   factor de atencion                 5611 non-null   object
 9   signos alcohol                     5611 non-null   object
 10  signos drogas                      5611 non-null   object
 11  casco                              5611 non-null   object
 12  cinturon de

- Buscamos correlaciones/reglas de asociación con el objetivo de identificar, en función del tipo de víctima, qué variables pueden dar una mayor importancia.
- Recordar que debemos estandarizar las columnas categoricas a númercas. Para ello utilizaremos el OneHotEncoder. Para la columna edad, estandarizaremos los valores con el MinMaxScaler.

In [22]:
# Separamos en target y features para realizar nuestro modelo
features = df_conductores.drop(columns = "tipo de victima")
target = df_conductores["tipo de victima"]

In [23]:
target = target.reset_index().drop("index", axis = 1)

In [24]:
# Usamos OneHotEncoder para convertir las variables categóricas en formato binario
encoder = OneHotEncoder(drop = "if_binary", sparse_output = False)

encoded_features  = encoder.fit_transform(features)

In [25]:
# Convertimos a DataFrame los datos codificados
df_cod_fin = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(features.columns))

In [26]:
# Se añade nuestro target al dataframe
df_cod_fin['tipo de victima'] = target

## Realizamos el análisis en función de tipo de víctima

In [27]:
df_cod_fin["tipo de victima"].unique()

array(['se desconoce', 'victima leve', 'ileso', 'victima grave',
       'victima mortal'], dtype=object)

In [28]:
def dividir_column(df, column):
    # Inicializa las columnas a 0
    df["victima mortal"] = 0
    df["victima grave"] = 0
    df["victima leve"] = 0
    df["ileso"] = 0
    df["se desconoce"] = 0

    # Usando np.where para asignar valores de forma condicional
    df["victima mortal"] = np.where(df[column] == "victima mortal", 1, 0)
    df["victima grave"] = np.where(df[column] == "victima grave", 1, 0)
    df["victima leve"] = np.where(df[column] == "victima leve", 1, 0)
    df["ileso"] = np.where(df[column] == "ileso", 1, 0)
    df["se desconoce"] = np.where(~df[column].isin(["victima mortal", "victima grave", "victima leve", "ileso"]), 1, 0)
    
    return df

In [29]:
dividir_column(df_cod_fin, "tipo de victima");

In [30]:
df_cod_global = df_cod_fin.drop(columns = "tipo de victima", axis = 1)

In [31]:
df_cod_global = df_cod_global.replace({0: False, 1: True})

C:\Users\alejandro.gc\AppData\Local\Temp\ipykernel_29548\3144679816.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cod_global = df_cod_global.replace({0: False, 1: True})


In [32]:
df_cod_global.head()

,presunto responsable_NO,presunto responsable_Se desconoce,presunto responsable_SÍ,presunto error_Ejecución incorrecta de maniobra o maniobra inadecuada,"presunto error_Indecisión, demora o retraso en tomar una decisión",presunto error_No entender una señal de tráfico o confundirla,presunto error_No se aprecian errores,presunto error_No ver un vehículo/peatón/obstáculo...,presunto error_No ver una señal,"presunto error_Olvidos (intermitentes, luces...)",...,tipo de vehiculo_furgoneta,tipo de vehiculo_maquinaria obra agricola,tipo de vehiculo_motocicleta,tipo de vehiculo_otro,tipo de vehiculo_turismo,victima mortal,victima grave,victima leve,ileso,se desconoce
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1,False,False,True,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,True,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
3,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
4,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np

# Función para calcular los itemsets frecuentes
def compute_frequent_itemsets(df, min_support):
    return apriori(df, min_support=min_support, use_colnames=True)

# Función para calcular las reglas de asociación
def compute_association_rules(frequent_itemsets, metric, min_threshold):
    return association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)

# Dividir el DataFrame en partes para procesar en paralelo
def chunk_data(df, n_chunks):
    return np.array_split(df, n_chunks)

In [34]:
import os 

n_cores = os.cpu_count()
n_cores

16

In [ ]:
from joblib import Parallel, delayed

# Soporte mínimo que debe tener un conjunto de items/variables para ser considerado frecuente
min_support = 0.1

# Confidence mide la probabilidad de que un item/variable esté presente en una 
# transacción dado otro conjunto de ítems/variables presentes
metric = "confidence"

# Utilizado para establecer un umbral mínimo para la métrica, significa que solo 
# se mantendrán aquellas reglas de asociación cuya confianza sea mayor o igual al X %
min_threshold = 0.1

# Dividir los datos en n partes
data_chunks_glob = chunk_data(df_cod_global, 4)

# Calcular los itemsets en paralelo
frequent_itemsets_list = Parallel(n_jobs=-1)(
    delayed(compute_frequent_itemsets)(chunk, min_support) for chunk in data_chunks_glob
)

# Combinar los resultados de los itemsets frecuentes
frequent_itemsets_combined = pd.concat(frequent_itemsets_list, ignore_index=True)

# Generar las reglas de asociación
rules_glob = compute_association_rules(frequent_itemsets_combined, metric, min_threshold)

In [343]:
rules_glob["antecedent_len"] = rules_glob["antecedents"].apply(lambda x: len(x))

In [344]:
rules_glob_vict = rules_glob[rules_glob['consequents'].str.contains("victima", case=False, na=False)]

In [345]:
rules_glob_vict

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len


In [346]:
rules_glob[(rules_glob['antecedent_len'] >= 5) &
           (rules_glob['confidence'] > 0.7)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len
27621,"(accion del peaton_Peaton no interviene, rango...",(posicion en el vehiculo_Se desconoce),0.469330,1.000000,0.469330,1.000000,1.000000,0.000000,inf,0.000000,5
27622,"(accion del peaton_Peaton no interviene, rango...",(presunta infraccion del peaton_Peaton no inte...,0.469330,1.000000,0.469330,1.000000,1.000000,0.000000,inf,0.000000,5
27623,"(accion del peaton_Peaton no interviene, rango...",(presunto responsable_NO),0.562054,0.536377,0.469330,0.835025,1.556789,0.167857,2.810271,0.816659,5
27624,"(accion del peaton_Peaton no interviene, rango...",(presunto error_No se aprecian errores),0.529957,0.582739,0.469330,0.885599,1.519718,0.160503,3.647353,0.727558,5
27625,"(accion del peaton_Peaton no interviene, presu...",(rango alcohol_No prueba/no positivo),0.474322,0.914408,0.469330,0.989474,1.082092,0.035605,8.131241,0.144317,5
...,...,...,...,...,...,...,...,...,...,...,...
152254,"(se desconoce, presunta infraccion del peaton_...","(tipo de vehiculo_turismo, accion del peaton_P...",0.539230,0.606990,0.407989,0.756614,1.246501,0.080682,1.614758,0.429182,5
152255,"(se desconoce, presunta infraccion del peaton_...","(tipo de vehiculo_turismo, accion del peaton_P...",0.522825,0.626961,0.407989,0.780355,1.244661,0.080198,1.698368,0.411942,5
152256,"(se desconoce, presunta infraccion del peaton_...","(tipo de vehiculo_turismo, accion del peaton_P...",0.559914,0.605563,0.407989,0.728662,1.203280,0.068925,1.453674,0.383876,5
152257,"(se desconoce, presunta infraccion del peaton_...","(tipo de vehiculo_turismo, accion del peaton_P...",0.532810,0.626961,0.407989,0.765730,1.221334,0.073937,1.592342,0.387901,5


In [347]:
rules_glob["consequents"].str.contains("victima", case=False, na=False).value_counts()

consequents
False    152435
Name: count, dtype: int64

## Analizamos víctimas mortales

In [348]:
df_cond_mort = df_cod_fin[df_cod_fin["tipo de victima"] == "victima mortal"]

In [349]:
df_cond_mort = df_cond_mort.drop(columns = "tipo de victima", axis = 1)

In [350]:
# Definir parámetros
min_support = 0.6
metric = "confidence"
min_threshold = 0.7

# Dividir los datos en 4 partes (ajusta según la cantidad de núcleos y tamaño de tus datos)
data_chunks_mort = chunk_data(df_cond_mort, 4)

c:\env_python\data_env\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [351]:
from joblib import Parallel, delayed

# Calcular los itemsets en paralelo
frequent_itemsets_list = Parallel(n_jobs=-1)(
    delayed(compute_frequent_itemsets)(chunk, min_support) for chunk in data_chunks_mort
)

# Combinar los resultados de los itemsets frecuentes
frequent_itemsets_combined = pd.concat(frequent_itemsets_list, ignore_index=True)

# Generar las reglas de asociación
rules_mort = compute_association_rules(frequent_itemsets_combined, metric, min_threshold)

In [352]:
rules_mort["antecedent_len"] = rules_mort["antecedents"].apply(lambda x: len(x))

In [353]:
rules_mort[(rules_mort['antecedent_len'] >= 7) &
           (rules_mort['confidence'] > 0.6)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len
20027,"(accion del peaton_Peaton no interviene, rango...",(caracteristica del permiso_En Vigor),0.727273,1.000000,0.727273,1.0,1.0,0.000000,inf,0.000000,7
20028,"(accion del peaton_Peaton no interviene, rango...",(posicion en el vehiculo_Se desconoce),0.727273,1.000000,0.727273,1.0,1.0,0.000000,inf,0.000000,7
20029,"(accion del peaton_Peaton no interviene, rango...",(sexo_Hombre),0.727273,0.909091,0.727273,1.0,1.1,0.066116,inf,0.333333,7
20030,"(accion del peaton_Peaton no interviene, rango...",(presunta infraccion del peaton_Peaton no inte...,0.727273,1.000000,0.727273,1.0,1.0,0.000000,inf,0.000000,7
20031,"(accion del peaton_Peaton no interviene, rango...",(victima mortal),0.727273,1.000000,0.727273,1.0,1.0,0.000000,inf,0.000000,7
...,...,...,...,...,...,...,...,...,...,...,...
72902,"(accion del peaton_Peaton no interviene, rango...",(presunta infraccion del peaton_Peaton no inte...,0.636364,1.000000,0.636364,1.0,1.0,0.000000,inf,0.000000,7
72903,"(accion del peaton_Peaton no interviene, rango...",(sexo_Hombre),0.636364,0.909091,0.636364,1.0,1.1,0.057851,inf,0.250000,7
72904,"(accion del peaton_Peaton no interviene, rango...",(victima mortal),0.636364,1.000000,0.636364,1.0,1.0,0.000000,inf,0.000000,7
72905,"(accion del peaton_Peaton no interviene, victi...",(rango alcohol_No prueba/no positivo),0.636364,1.000000,0.636364,1.0,1.0,0.000000,inf,0.000000,7
